In [ ]:
import os
import cv2
import numpy as np
from deepface import DeepFace
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import joblib
from scipy.spatial.distance import cosine
import time

MODEL_PATH = "deepface_recognition_model.pkl"
EMBEDDINGS_PATH = "training_embeddings.pkl"

In [ ]:

# ---------- TRAINING ----------
def load_dataset(dataset_path):
    X, y = [], []
    for person_name in os.listdir(dataset_path):
        person_folder = os.path.join(dataset_path, person_name)
        if not os.path.isdir(person_folder):
            continue

        print(person_name+"/n")
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            try:
                embedding = DeepFace.represent(img_path=img_path, model_name='Facenet', enforce_detection=True)[0]["embedding"]
                X.append(embedding)
                y.append(person_name)
            except:
                continue
    return np.array(X), np.array(y)


def train_model(dataset_path):
    X, y = load_dataset(dataset_path)
    print(f"[INFO] Training on {len(X)} samples from {len(set(y))} people")

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    clf = SVC(kernel='linear', probability=True)
    clf.fit(X, y_encoded)

    # Save SVM + label encoder
    joblib.dump((clf, label_encoder), MODEL_PATH)
    # Save embeddings for distance check
    joblib.dump((X, y), EMBEDDINGS_PATH)
    print("[INFO] Model and embeddings saved!")


In [32]:
from collections import deque, Counter
import cv2, os, joblib
import numpy as np
from deepface import DeepFace
from scipy.spatial.distance import cosine
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

# ---------- LIVE RECOGNITION ----------
def recognize_face_live(prob_threshold=0.95, dist_threshold=0.25):
    clf, label_encoder = joblib.load(MODEL_PATH)
    known_embeddings, known_labels = joblib.load(EMBEDDINGS_PATH)

    cap = cv2.VideoCapture(0)
    frame_count = 0

    # === (1) Rolling detection buffer for stable recognition ===
    recent_detections = deque(maxlen=5)
    confirmed_person = None

    while True:
        ret, frame = cap.read()
        if not ret:
            continue

        frame_count += 1

        # === (2) Frame skipping optimization ===
        if frame_count % 20 != 0:
            # === (4) Live overlay feedback ===
            if confirmed_person:
                cv2.putText(frame, f"✅ Confirmed: {confirmed_person}",
                            (40, 60), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Scanning...",
                            (40, 60), cv2.FONT_HERSHEY_DUPLEX, 1.2, (0, 255, 255), 2)

            cv2.imshow("Live Face Recognition", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            continue

        # Resize frame for faster DeepFace
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        temp_path = "temp_frame.jpg"
        cv2.imwrite(temp_path, small_frame)

        try:
            embeddings = DeepFace.represent(img_path=temp_path,
                                            model_name='Facenet',
                                            enforce_detection=True)
        except:
            embeddings = []

        for emb_dict in embeddings:
            enc = emb_dict["embedding"]

            # SVM prediction
            probs = clf.predict_proba([enc])[0]
            max_prob = np.max(probs)
            pred = clf.predict([enc])[0]
            name = label_encoder.inverse_transform([pred])[0]
            #print(name)

            # Distance check
            distances = [cosine(enc, e) for e in known_embeddings]
            min_dist = min(distances)

            if (max_prob < prob_threshold and min_dist > dist_threshold):
                name = "Unknown"

            
            # === 💬 Print live detection info ===
            print(f"[DETECTION] Frame {frame_count}: {name} , dist={min_dist:.3f})")

            # === (1) Stability filter update ===
            recent_detections.append(name)
            name_counts = Counter(recent_detections)
            most_common_name, freq = name_counts.most_common(1)[0]

            if freq >= 5 and most_common_name != "Unknown":
                confirmed_person = most_common_name
                print(f"\n✅ Confirmed identity: {confirmed_person}")
                cap.release()
                cv2.destroyAllWindows()
                return f"YES: Recognised {confirmed_person}"

            # === (4) Overlay name feedback ===
            cv2.putText(frame, name, (40, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2,
                        (0, 255, 0) if name != "Unknown" else (0, 0, 255), 2)

        cv2.imshow("Live Face Recognition", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return "NO FACE"


In [34]:
train_model(r"C:\Users\kusha\Downloads\dataset")

Bapat bro/n
Bhavishya/n
Bijlani/n
Jay/n
Kushaal/n
[INFO] Training on 17 samples from 5 people
[INFO] Model and embeddings saved!


In [39]:
# ---------- MAIN ----------
if __name__ == "__main__":
    result = recognize_face_live()
    print(result)

[DETECTION] Frame 20: Kushaal , dist=0.059)
[DETECTION] Frame 20: Kushaal , dist=0.088)
[DETECTION] Frame 40: Bhavishya , dist=0.220)
[DETECTION] Frame 100: Kushaal , dist=0.144)
[DETECTION] Frame 120: Kushaal , dist=0.116)
[DETECTION] Frame 140: Kushaal , dist=0.066)
[DETECTION] Frame 160: Kushaal , dist=0.171)
[DETECTION] Frame 180: Unknown , dist=0.416)
[DETECTION] Frame 220: Unknown , dist=0.441)
[DETECTION] Frame 240: Kushaal , dist=0.188)
[DETECTION] Frame 260: Unknown , dist=0.301)
[DETECTION] Frame 300: Unknown , dist=0.413)
[DETECTION] Frame 320: Kushaal , dist=0.040)
[DETECTION] Frame 340: Kushaal , dist=0.063)
[DETECTION] Frame 360: Kushaal , dist=0.157)
[DETECTION] Frame 380: Kushaal , dist=0.110)
[DETECTION] Frame 400: Unknown , dist=0.479)
[DETECTION] Frame 440: Bijlani , dist=0.138)
[DETECTION] Frame 460: Unknown , dist=0.468)
[DETECTION] Frame 480: Kushaal , dist=0.196)
[DETECTION] Frame 500: Bhavishya , dist=0.222)
[DETECTION] Frame 520: Unknown , dist=0.338)
[DETECTIO